<a href="https://colab.research.google.com/github/piratesdragon/homework_actual_problems/blob/main/homework_gpt.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Домашнее задание № 10. Генерация текста

### Задание 1 (8 баллов).

Попробуйте дообучать GPT на каком-то другом тексте (можете попробовать любые стихи или какие-то специфичные вещи вроде анекдотов или репа). 
Попробуйте разные методы и параметры генерации (beam search, температура, top_k и тп). Сохраните в тетрадке несколько хороших сгенерированных текстов


In [ ]:
%pip install transformers

In [ ]:
from transformers.utils import logging
logging.set_verbosity(40)

from transformers import TextDataset, DataCollatorForLanguageModeling, Trainer, TrainingArguments


from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("cuda:0")

In [ ]:
# Модель от сбера
model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

In [ ]:
# Вредные советы Григория Остера :)

text = """
	Потерявшийся ребенок
     Должен помнить, что его
     Отведут домой, как только
     Назовет он адрес свой.
     Надо действовать умнее,
     Говорите: "Я живу Возле пальмы с обезьяной На далеких островах".
     Потерявшийся ребенок,
     Если он не дурачок,
     Не упустит верный случай
     В разных странах побывать.

     Рукам никогда нигде
     Не трогай ничего.
     Не впутывайся ни во что
     И никуда не лезь.
     В сторонку молча отойди,
     Стань скромно в уголке
     И тихо стой, не шевелясь,
     До старости своей.

     Кто не прыгал из окошка
     Вместе с маминым зонтом,
     Тот лихим парашютистом
     Не считается пока.
     Не лететь ему, как птице,
     Над взволнованной толпой,
     Не лежать ему в больнице
     С забинтованной ногой.

     Если всей семьей купаться
     Вы отправились к реке,
     Не мешайте папе с мамой
     Загорать на берегу.
     Не устраивайте крика,
     Дайте взрослым отдохнуть.
     Ни к кому не приставая,
     Постарайтесь утонуть.


     Нет приятнее занятья,
     Чем в носу поковырять.
     Всем ужасно интересно,
     Что там спрятано внутри.
     А кому смотреть противно,
     Тот пускай и не глядит.
     Мы же в нос к нему не лезем,
     Пусть и он не пристает.
     Если вас поймала мама
     За любимым делом вашим,
     Например, за рисованьем
     В коридоре на обоях,
     Объясните ей, что это -
     Ваш сюрприз к Восьмому марта.
     Называется картина:
     "Милой мамочки портрет".


     Не бери чужое, если
     На тебя глядят чужие.
     Пусть они глаза закроют
     Или выйдут на часок.
     А своих чего бояться!
     Про своих свои не скажут.
     Пусть глядят. Хватай чужое
     И тащи его к своим.


     Никогда вопросов глупых
     Сам себе не задавай,
     А не то еще глупее
     Ты найдешь на них ответ.
     Если глупые вопросы
     Появились в голове,
     Задавай их сразу взрослым.
     Пусть у них трещат мозги.


     Посещайте почаще
     Театральный буфет.
     Там пирожные с кремом,
     С пузырьками вода.
     Как дрова на тарелках
     Шоколадки лежат,
     И сквозь трубочку можно
     Пить молочный коктейль.
     Не просите билеты
     На балкон и в партер,
     Пусть дадут вам билеты
     В театральный буфет.
     Уходя из театра,
     Унесете с собой
     Под трепещущим сердцем,
     В животе, бутерброд.


     Родился девочкой - терпи
     Подножки и толчки.
     И подставляй косички всем,
     Кто дернуть их не прочь.
     Зато когда-нибудь потом
     Покажешь кукиш им
     И скажешь: "Фигушки, за вас
     Я замуж не пойду!"



     Если вы с друзьями вместе
     Веселитесь во дворе,
     А с утра на вас надели
     Ваше новое пальто,
     То не стоит ползать в лужах
     И кататься по земле,
     И взбираться на заборы,
     Повисая на гвоздях.
     Чтоб не портить и не пачкать Ваше новое пальто,
     Нужно сделать его старым.
     Это делается так:

     Залезайте прямо в лужу,
     Покатайтесь по земле,
     И немножко на заборе
     Повисите на гвоздях.
     Очень скоро станет старым
     Ваше новое пальто,
     Вот теперь спокойно можно
     Веселиться во дворе.
     Можно смело ползать в лужах
     И кататься по земле,
     И взбираться на заборы,
     Повисая на гвоздях.


     Если вы по коридору
     Мчитесь на велосипеде,
     А навстречу вам из ванной
     Вышел папа погулять,
     Не сворачивайте в кухню,
     В кухне - твердый холодильник.
     Тормозите лучше в папу.
     Папа мягкий. Он простит.


     Если вас навек сплотили,
     Озарили и ведут,
     Не пытайтесь уклониться
     От движенья к торжеству.
     Все равно на труд поднимет
     И на подвиг вдохновит
     Вас великий и могучий,
     И надежный наш оплот.


     Главным делом жизни вашей
     Может стать любой пустяк.
     Надо только твердо верить,
     Что важнее дела нет.
     И тогда не помешает
     Вам ни холод, ни жара,
     Задыхаясь от восторга,
     Заниматься чепухой.


     Бейте палками лягушек.
     Это очень интересно.
     Отрывайте крылья мухам,
     Пусть побегают пешком.
     Тренируйтесь ежедневно,
     И наступит день счастливый -
     Вас в какое-нибудь царство
     Примут главным палачом.

     Девчонок надо никогда
     Нигде не замечать.
     И не давать прохода им
     Нигде и никогда.
     Им надо ножки подставлять,
     Пугать из-за угла,
     Чтоб сразу поняли они:
     До них вам дела нет.
     Девчонку встретил - быстро ей
     Показывай язык.
     Пускай не думает она,
     Что ты в нее влюблен.


     Начиная драку с папой,
     Затевая с мамой бой,
     Постарайся сдаться маме, -
     Папа пленных не берет.
     Кстати, выясни у мамы,
     Не забыла ли она -
     Пленных бить ремнем по попе
     Запрещает Красный Крест.


     Если ты весь мир насилья
     Собираешься разрушить,
     И при этом стать мечтаешь
     Всем, не будучи ничем,
     Смело двигайся за нами
     По проложенной дороге,
     Мы тебе дорогу эту
     Можем даже уступить.


     Не соглашайся ни за что
     Ни с кем и никогда,
     А кто с тобой согласен, тех
     Трусливыми зови.
     За это все тебя начнут
     Любить и уважать.
     И всюду будет у тебя
     Полным полно друзей.


     Если в кухне тараканы
     Маршируют по столу,
     И устраивают мыши
     На полу учебный бой,
     Значит, вам пора на время
     Прекратить борьбу за мир,
     И все силы ваши бросить
     На борьбу за чистоту.


     Если вы собрались другу
     Рассказать свою беду,
     Брать за пуговицу друга
     Бесполезно - убежит,
     И на память вам оставит
     Эту пуговицу друг.
     Лучше дать ему подножку,
     На пол бросить, сверху сесть
     И тогда уже подробно
     Рассказать свою беду.

     Если ты пришел к знакомым,
     Не здоровайся ни с кем.
     Слов: "пожалуйста", "спасибо"
     Никому не говори.
     Отвернись и на вопросы
     Ни на чьи не отвечай.
     И тогда никто не скажет
     Про тебя, что ты болтун.

"""

In [ ]:
train_path = 'train_dataset.txt'
with open(train_path, "wb") as f:
  f.write(text.encode('utf-8'))

train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)

data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

/usr/local/lib/python3.9/dist-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [ ]:
training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=100, 
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=16, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

In [ ]:
trainer.train()

{'train_runtime': 53.9622, 'train_samples_per_second': 46.329, 'train_steps_per_second': 1.853, 'train_loss': 0.09425873756408691, 'epoch': 100.0}


TrainOutput(global_step=100, training_loss=0.09425873756408691, metrics={'train_runtime': 53.9622, 'train_samples_per_second': 46.329, 'train_steps_per_second': 1.853, 'train_loss': 0.09425873756408691, 'epoch': 100.0})

In [ ]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=300,
                        repetition_penalty=3.5
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Дождь идет 
     По лужам и лужам,
     Повисая на гвоздях.
     Если вы с друзьями вместе
     Веселитесь во дворе,
     Не мешайте им веселиться.
     Пусть они веселятся со своим папой
     Вместе с вами.


     Если у вас в квартире тараканы
     Маршируют по потолку,
     То не стоит ползать в лужах.
     Лучше займитесь генеральной уборкой
     Вашей новой квартиры.
     Это избавит вас от необходимости каждый день
     Убираться в своей квартире.
     А заодно и от лишних хлопот.


     Если вы все время куда-то торопитесь,
     То лучше всего пешком
     Доехать до ближайшей железнодорожной станции.
     Там вам предложат какой-нибудь интересный
     Экскурсионный маршрут.
     Например, по железнодорожному полотну
     Вы покатаетесь на велосипеде.
     А потом пройдете несколько остановок
     В разных концах города.
     Можно даже взять напрокат велосипед.
     Кстати, если вы всей семьей
     Прогуляетесь по заснеженному полю
     Под открытым небом, то обязательно
     В

Текст с параметрами из семинара звучит неплохо, но во втором "стишке" фразы почти неизмененные из оригинального текста(


In [ ]:
# уменьшила top_k, вместо repetition penalty запретила повторять биграммы

text = "Если ты не хочешь в школу"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, 
                        top_k=30,
                        max_length=150,
                        no_repeat_ngram_size=2
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Если ты не хочешь в школу опоздать,
     Не ходи туда, где тебя ждут.     Там тебя никто не ждет.

Прогуливаясь по Невскому проспекту, вы заметили на бортике фонтана трех лягушек. Подбежав к ним, лягушки спрятались под скамейку и стали ждать, пока вы их позовете на помощь. Но когда вы подошли к лягушкам поближе, то увидели, что они уже не прячутся, а ползут к вам на брюках. Помогите им выбраться из лужи. Это очень просто. Нужно просто сказать: "Лягушки, вылезайте на берег, я вас оттуда достану". И тогда вы сразу же почувствуете себя уверенными в своих силах


Мне очень нравятся оба варианта)

In [ ]:
# увеличила число пучков, немного увеличила top k

text = "Если ты"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=7, 
                        top_k=40,
                        max_length=150,
                        no_repeat_ngram_size=2
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Если ты не с нами,
     Не с теми, за кого себя выдаешь.     И если с тобой что-то не так - 
Повисай на гвоздях до скончания века.

Не устраивай крика, не кричи: "Я с вами, потому что мы рядом" - и все у тебя будет в порядке. 

Тебе кажется, что ты один на всем белом свете.  А на самом деле вокруг тебя сплошные враги и завистники. Отведи глаза в сторону и улыбнись им. И тогда они обязательно улыбнутся тебе в ответ. А еще лучше - просто кивни им головой, как это делают дети, когда им показывают мультики про папу Карло и маму


Здесь, кажется, довольно связно, но пугающе

In [ ]:
# убрала пучки, немного увеличила top k, запретила повторять триграммы

text = "Если ты"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        top_k=60,
                        max_length=150,
                        temperature = 0.4,
                        no_repeat_ngram_size=3
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Если ты с мамой вместе,
     Не мешай папе с мамой жить.
     И вообще, не мешай маме жить,
    А то папа с мамой поссорились.


     Если ты с папой вместе, -
     Учись хорошо плавать,
   А не то утонешь.
   И вообще - не мешайте папе
     Заниматься ерундой.
    Папа с мамой любят поговорить
     О пустяках.
  Если ты папе не веришь,
         Объясни ему, что это -
         Папа тебе маму испортил.
         И вообще:
     Папа всех переплюнет.
 

     Никогда не спорь с папой
     Ни за что. Он очень хитрый


Кажется, даже немного похоже по смыслу на вредные советы)

In [ ]:

text = "Если вам слегка за двадцать"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        top_k=60,
                        max_length=150,
                        temperature = 0.4,
                        no_repeat_ngram_size=3
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Если вам слегка за двадцать,
     Не стоит отчаиваться.
     Вам помогут друзья.


     Если вы на работе одни,
    Не устраивайте крика,
   Дайте спокойно отдохнуть.
    И спокойно все вокруг
     Переварите.
   И спокойно на работе
     Повисайте.
  И не мешайте работе
    До победного конца.
 

     Когда вам немного за тридцать,
  Не стоит расстраиваться.
         Это всего лишь возраст.
                 Может статься, что вы
     Совсем не изменились.
_______________

  Если вам немного под сорок,
         Не стоит впадать в панику.
Потерявшийся ребенок



Ну и немного позитива от модели)

### Задание  2 (2 балла)

Ответьте на следующие вопросы:

1) В каких статья были представлены GPT-1, GPT-2, GPT-3?

2) Как собирался обучающий корпус для GPT-3? Каким образом создатели старались обеспечить высокое качество текстов в обучающей выборке?

**Вопрос 1**
- GPT-1: Improving Language Understanding by Generative Pre-Training (2018) (https://cdn.openai.com/research-covers/language-unsupervised/language_understanding_paper.pdf)
- GPT-2: Language Models are Unsupervised Multitask Learners (2019) (https://cdn.openai.com/better-language-models/language_models_are_unsupervised_multitask_learners.pdf)
- GPT-3: Language Models are Few-Shot Learners (2020) (https://arxiv.org/pdf/2005.14165.pdf)



**Вопрос 2**

Основной датасет, на котором обучался GPT-3 - Common Crawl, состоящий практически из триллиона токенов. Он был создан путем многолетнего (с 2008 года) веб-краулинга огромного числа страниц в интернете.

Чтобы повысить качество текстов, подающихся на обучение модели, этот датасет был отфильтрован: были взяты несколько менее крупных, но качественных корпусов, таких как корпус англоязычной википедии и книжные корпуса, и Common Crawl был отфильтрован на основе сходства с ними. Эти корпуса также были добавлены в обучающую выборку. 

Кроме того, к корпусу на уровне документов была применена дедупликация, чтобы предотвратить повторы и избыточность, а данные брались не за все время, а только за период 2016-2019 годов. Итоговый объем копуса: 410 миллиардов токенов.


